## 1. libraries import and setup

In [1]:
from pathlib import Path
import importlib.util
import sys
import rioxarray as rxr
import tensorflow as tf
from matplotlib import pyplot as plt
import multiprocessing
import numpy as np
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
from tensorflow import keras
from keras.layers import Input, Conv2D
from keras.models import Model
import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [2]:
# load the setup.py module
path = Path.cwd().parent / 'src' / 'setup.py'
spec = importlib.util.spec_from_file_location('setup', path)
setup = importlib.util.module_from_spec(spec)
spec.loader.exec_module(setup)

# call the make_folders function
raw_data_dir, processed_data_dir, output_dir = setup.make_folders()

created directory data\raw_data\DOPs
created directory data\raw_data\laz_ALS
created directory data\raw_data\laz_DAP
created directory data\raw_data\dtm_tiles
created directory data\raw_data\test_tiles
created directory data\processed_data\DOPs
created directory data\processed_data\nDSMs_ALS
created directory data\processed_data\nDSMs_DAP
created directory data\processed_data\laz_ALS
created directory data\processed_data\laz_DAP
created directory data\processed_data\datasets
created directory data\processed_data\models
created directory data\processed_data\gap_polygons_ALS
created directory data\processed_data\gap_polygons_DAP
created directory data\metadata
created directory src
created directory scripts
created directory output


## 2. data reading and preparation

In [4]:
# read preprocessed dataset (TDOP + nDSM for training with gap mask)
train_ds = rxr.open_rasterio(processed_data_dir / 'datasets' / 'train_ds.tif',
                             band_as_variable=True)

# use the long_names as variable names 
# (previously assigned in script 'data_preparation')
for var_name in train_ds.data_vars:
    long_name = train_ds[var_name].attrs['long_name']
    train_ds = train_ds.rename({var_name: long_name})

train_ds

<xarray.Dataset>
Dimensions:      (x: 7000, y: 10501)
Coordinates:
  * x            (x) float64 5.445e+05 5.445e+05 5.445e+05 ... 5.48e+05 5.48e+05
  * y            (y) float64 5.733e+06 5.733e+06 ... 5.728e+06 5.728e+06
    spatial_ref  int32 0
Data variables:
    red          (y, x) float32 ...
    green        (y, x) float32 ...
    blue         (y, x) float32 ...
    nir          (y, x) float32 ...
    ndsm         (y, x) float32 ...
    gap_mask     (y, x) float32 ...
Attributes:
    AREA_OR_POINT:  Area

In [5]:
# import data generator
src_dir = Path.cwd().parent / 'src'
sys.path.append(str(src_dir))
from cidg import CustomImageDataGenerator

# define tile size
tilesize = 224

# create an instance of the data generator
# hand over the training dataset (TDOP + nDSM with gap mask)
cidg_training   = CustomImageDataGenerator(train_ds, tilesize, sampletype='training')
cidg_validation = CustomImageDataGenerator(train_ds, tilesize, sampletype='validation')

In [6]:
# get preprocessing of desired backbone
preprocess_input = sm.get_preprocessing('resnet34')

# preprocess input
cidg_train_preprocessed = preprocess_input(cidg_training)
cidg_val_preprocessed = preprocess_input(cidg_validation)

## 3. model initialization and training

In [6]:
# define number of input channels (RGBI + nDSM)
channels = 5

# define model:
# U-Net with ResNet 34 as backbone and pretrained weights
base_model = sm.Unet(
    backbone_name='resnet34',
    encoder_weights='imagenet')

# add extra convolution layer to map N -> 3 channels data 
# and train with pretrained weights
# --> see documentation of the segmentation_models library 
# (https://segmentation-models.readthedocs.io/en/latest/tutorial.html?highlight=preprocessing#training-with-non-rgb-data)
inp = Input(shape=(tilesize, tilesize, channels))
l1 = Conv2D(3, (1, 1))(inp) # map N channels data to 3 channels
out = base_model(l1)

model = Model(inp, out, name=base_model.name)

# compile with Adam optimizer and Binary Cross Entropy
# use Intersection over Union (IoU) 
# and F1-score as accuarcy measures
model.compile(optimizer='adam',
              loss=sm.losses.BinaryCELoss(),
              metrics=[sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)])

In [ ]:
# implement stop of training when validation loss
# has stopped improving for 10 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1-score',
    patience=10,
    restore_best_weights=True,
    mode='max'
)

# callback to save the best model
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    str(processed_data_dir / 'models' / 'resnet34_unet_rgbi_ndsm'), 
    monitor='val_f1-score', 
    verbose=1, 
    save_best_only=True,
    mode='max'
)

# callback to reduce the learning rate
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_f1-score', 
    factor=0.2, 
    patience=5, 
    min_lr=0.00001, 
    verbose=1,
    mode='max'
)

# training
model_history = model.fit(cidg_train_preprocessed,
                          validation_data=cidg_val_preprocessed,
                          epochs=100,
                          initial_epoch=0,
                          callbacks=[early_stopping, checkpoint, reduce_lr],
                          workers=multiprocessing.cpu_count() - 4)

In [9]:
# get epoch with the best model (based on validation loss)
best_model_epoch = np.argmax(model_history.history['val_f1-score'])
epoch_number = best_model_epoch + 1

# print training IoU for the best epoch
print(f"Training IoU best model at epoch {epoch_number}: {round(model_history.history['iou_score'][best_model_epoch], 2)}")
# print validation IoU for the best epoch
print(f"Validation IoU best model at epoch {epoch_number}: {round(model_history.history['val_iou_score'][best_model_epoch], 2)}")
# print training F1-score for the best epoch
print(f"Training F1-score best model at epoch {epoch_number}: {round(model_history.history['f1-score'][best_model_epoch], 2)}")
# print validation F1-score for the best epoch
print(f"Validation F1-score best model at epoch {epoch_number}: {round(model_history.history['val_f1-score'][best_model_epoch], 2)}")
# print training loss for the best epoch
print(f"Training loss best model at epoch {epoch_number}: {round(model_history.history['loss'][best_model_epoch], 2)}")
# print validation loss for the best epoch
print(f"Validation loss best model at epoch {epoch_number}: {round(model_history.history['val_loss'][best_model_epoch], 2)}")

Training IoU best model at epoch 19: 0.67
Validation IoU best model at epoch 19: 0.48
Training F1-score best model at epoch 19: 0.8
Validation F1-score best model at epoch 19: 0.64
Training loss best model at epoch 19: 0.05
Validation loss best model at epoch 19: 0.14
